In [ ]:
import numpy as np
import pickle


import implementations as imp
import proj1_helpers as ph
import glove as gl

from nltk import pos_tag
#nltk.download()
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import fasttext

In [ ]:

smile = pickle.load(open('twitter-datasets/train_smile.pkl', 'rb'))
smile = [int((x+1)/2) for x in smile]
smile[0:10]

In [ ]:
smile[-10:]

In [ ]:
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
stopwords_stem = [ stemmer.stem(x) for x in stopwords.words('english')]

In [ ]:
tweets_train_nlp = pickle.load(open('tweets_nlp.pkl', 'rb'))

In [ ]:

list_word_with_rep = [  ' '.join([y  for y in x if x not in stopwords_stem]) for x in tweets_train_nlp ]


In [ ]:
list_word_with_rep[ 0]

In [ ]:
X = [ '__label__'+str(smile[i])+' ' + list_word_with_rep[i] + '\n' for i in range(len(list_word_with_rep))]
X[0]

In [ ]:
with open('fastest_train.txt', 'w', encoding="utf-8") as f:
        for x in X:
             f.write( x)
                
                


In [ ]:
clf = fasttext.supervised('fastest_train.txt' , 'model',epoch=10,label_prefix='__label__',  ws =10,
    thread=10
)


In [ ]:
clf 

In [ ]:
def ke_free_lemm (word , type_) :
    try :
        res = lemmatiser.lemmatize(word , type_)
        print(res,word ,type_)
        return 
        
    except KeyError :
        return word

def lemmatize (tweet):
    
    return [ lemmatiser.lemmatize(x) for x in tweet ]

def split_ok (tweet) :
    res = tweet[:-1].split(' ')
    res = [ x for x in res if x != '']
    return res


In [ ]:
def split_id_tweet (list_tweets) :
    ids = [x.split(',')[0] for x in list_tweets]
    tweets =  [','.join(x.split(',')[1:]) for x in list_tweets]
    return ids , tweets

In [ ]:
ids_test=[]
tweets_test=[]
with open('test_data.txt') as f :
    ids , tweets_test = split_id_tweet(f.readlines())
    
tweets_test_nlp = [ [ stemmer.stem(ke_free_lemm(word[0],word[1])) for word in pos_tag(split_ok (x))] \
                    for x in tweets_test  \
                    ]    



In [ ]:
tweets_test[:10]

In [ ]:
X_test =  [  ' '.join([y  for y in x if x not in stopwords_stem]) + '\n' for x in tweets_test_nlp ]
X_test[:10]


In [ ]:
with open('fastest_test.txt', 'w', encoding="utf-8") as f:
        for x in X_test:
             f.write( x+ '\n')

In [ ]:
X_test[0]

In [ ]:
labels = clf.predict(X_test )
#res = np.sign(res)
labels

In [ ]:
clf2 = clf.test('fastest_train.txt')
clf2.precision

In [ ]:
len(X_test[0])

In [ ]:
imp.create_csv_submission(ids,res,'results_nlp.csv')

In [ ]:
count = 0
for x in res :
    if x  == 0 :
        count +=1
count , len(res) , count/len(res)

In [ ]:
cf = fasttext.load_model('model.bin')
cf.labels
